<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv4/blob/main/adverserial_attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

download_links = ['https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2.npz',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_DNN_drebin_best.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/data/adverserial_attacks_functions.py',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM_weightedLoss.pth',
                  'https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/model_AT_rFGSM.pth',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/insertion_array.pkl',
                  'https://github.com/mostafa-ja/mal_adv4/raw/main/dataset/removal_array.pkl'
]

In [2]:
import gdown
output_filepath = '/content/'
for link in download_links:
  gdown.download(link, output_filepath)


Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_0.npz
To: /content/sparse_matrix_0.npz
100%|██████████| 461k/461k [00:00<00:00, 11.0MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_1.npz
To: /content/sparse_matrix_1.npz
100%|██████████| 148k/148k [00:00<00:00, 3.47MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_2.npz
To: /content/sparse_matrix_2.npz
100%|██████████| 150k/150k [00:00<00:00, 4.21MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y0.npz
To: /content/sparse_matrix_y0.npz
100%|██████████| 5.79k/5.79k [00:00<00:00, 14.4MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y1.npz
To: /content/sparse_matrix_y1.npz
100%|██████████| 2.64k/2.64k [00:00<00:00, 8.38MB/s]
Downloading...
From: https://github.com/mostafa-ja/mal_adv3/raw/main/drebin/sparse_matrix_y2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pickle
import random
import time

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from adverserial_attacks_functions import *

torch.manual_seed(0)



cuda
cuda


In [21]:
# Open the .pkl file
with open('/content/insertion_array.pkl', 'rb') as f:
    # Load the object
    insertion_array = pickle.load(f)

# Close the file
f.close()

insertion_array = torch.tensor(insertion_array).to(device)
print(len(insertion_array))

# Open the .pkl file
with open('/content/removal_array.pkl', 'rb') as f:
    # Load the object
    removal_array = pickle.load(f)

# Close the file
f.close()

removal_array = torch.tensor(removal_array).to(device)
print(len(removal_array))


10000
10000


In [5]:

# Load dataset
X_train = sparse.load_npz("/content/sparse_matrix_0.npz").toarray()
X_val = sparse.load_npz("/content/sparse_matrix_1.npz").toarray()
X_test = sparse.load_npz("/content/sparse_matrix_2.npz").toarray()

X_train = torch.tensor(X_train, dtype=torch.int8)
X_val = torch.tensor(X_val, dtype=torch.int8)
X_test = torch.tensor(X_test, dtype=torch.int8)


y_train = sparse.load_npz("/content/sparse_matrix_y0.npz").toarray().reshape((-1, 1))
y_val = sparse.load_npz("/content/sparse_matrix_y1.npz").toarray().reshape((-1, 1))
y_test = sparse.load_npz("/content/sparse_matrix_y2.npz").toarray().reshape((-1, 1))

y_train = torch.tensor(y_train, dtype=torch.int8)
y_val = torch.tensor(y_val, dtype=torch.int8)
y_test = torch.tensor(y_test, dtype=torch.int8)


print("Shapes:")
print("x_train:", X_train.shape)
print("x_val:", X_val.shape)
print("x_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

Shapes:
x_train: torch.Size([28683, 10000])
x_val: torch.Size([9562, 10000])
x_test: torch.Size([9562, 10000])
y_train: torch.Size([28683, 1])
y_val: torch.Size([9562, 1])
y_test: torch.Size([9562, 1])


In [6]:
# number of benigns and malicious sample in training dataset
n_ben = (y_train.squeeze()== 0).sum().item()
n_mal = (y_train.squeeze()== 1).sum().item()
print('the proportion of malwares : ', n_mal/(n_mal+n_ben))

# Combine features and labels into datasets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Define the DataLoader for training, validation, and test sets
batch_size = 2048
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Clear unnecessary variables
del train_dataset, val_dataset, test_dataset, y_train, y_val, y_test


the proportion of malwares :  0.11386535578565701


In [7]:
model_DNN = MalwareDetectionModel().to(device)
# Load model parameters
model_DNN.load_state_dict(torch.load('model_DNN_drebin_best.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [8]:
model_evaluation(model_DNN, test_loader, device)

Test Accuracy: 0.9912
Test balanced Accuracy: 0.9785
Precision: 0.9637
Recall: 0.9619
F1-score: 0.9628
True Positives (TP): 1087
True Negatives (TN): 8391
False Positives (FP): 41
False Negatives (FN): 43
False Negative Rate (FNR): 3.8053
False Positive Rate (FPR): 0.4862


In [9]:
def dfgsm_k(x, y, model, k=25, epsilon=0.02, alpha=1., initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False, device="cpu"):
    """
    FGSM^k with deterministic rounding
    :param y: ground truth labels
    :param x: feature vector
    :param model: neural network model
    :param k: number of steps
    :param epsilon: update value in each direction
    :param alpha: hyperparameter for controlling the portionate of rounding
    :param initial_rounding_threshold: threshold parameter for rounding the initial x_next
    :param round_threshold: threshold parameter for rounding
    :param is_report_loss_diff: flag to report loss difference
    :param is_sample: flag to sample randomly from the feasible area
    :return: the adversarial version of x according to dfgsm_k (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize starting point
    x_next = x.clone()
    x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

    # Multi-step
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        gradients = grad_vars[0].data
        grad4insertion = (gradients > 0) * gradients * (x_var <= 0.5)
        # Find the next sample
        x_next = x_next + epsilon * torch.sign(grad4insertion)

        # Projection
        x_next = torch.clamp(x_next, min=0., max=1.)

    # Rounding step
    if random:
       round_threshold = (torch.rand(x_next.size()) * alpha).to(device=x.device)
    x_next = round_x(x_next, round_threshold=round_threshold)

    # Feasible projection
    x_next = or_float_tensors(x_next, x)

    # Compute adversarial loss
    loss_adv = criterion(model(x_next), y.view(-1).long()).data

    if is_report_loss_diff:
        #print(f"Natural loss: {loss_natural.mean():.4f}, Adversarial loss: {loss_adv.mean():.4f}, Difference: {(loss_adv.mean() - loss_natural.mean()):.4f}")
        outputs = model(x_next)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"dFGSM: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    # Replace with natural if adversarial loss is higher
    replace_flag = (loss_adv < loss_natural).squeeze()
    x_next[replace_flag] = x[replace_flag]

    return x_next


In [11]:
# AT-rFGSM: Adversarial Taraining based on rFGSM attack
model_AT = MalwareDetectionModel().to(device)

attack_param = {"k":50, "epsilon":0.02, 'random':True, "is_sample":False, 'is_report_loss_diff':False}
adversarial_training(model_AT, train_loader, val_loader, adv_epochs=50, attack=dfgsm_k, **attack_param)

Epoch: 1/50 | Training loss (epoch level): 0.6947 | Train accuracy: 75.76%
	Val accuracy(without attack) 89.88% and accuracy(with attack) 12.65% under attack and overall accuracy 51.26%.
	Model select at epoch 1 with validation accuracy 51.26% and accuracy 12.65% under attack.
Epoch: 2/50 | Training loss (epoch level): 0.6497 | Train accuracy: 89.79%
	Val accuracy(without attack) 92.36% and accuracy(with attack) 15.88% under attack and overall accuracy 54.12%.
	Model select at epoch 2 with validation accuracy 54.12% and accuracy 15.88% under attack.
Epoch: 3/50 | Training loss (epoch level): 0.5433 | Train accuracy: 91.49%
	Val accuracy(without attack) 95.4% and accuracy(with attack) 42.11% under attack and overall accuracy 68.75%.
	Model select at epoch 3 with validation accuracy 68.75% and accuracy 42.11% under attack.
Epoch: 4/50 | Training loss (epoch level): 0.2668 | Train accuracy: 95.22%
	Val accuracy(without attack) 96.7% and accuracy(with attack) 72.76% under attack and overal

In [12]:
# Create an instance of your model
model_AT_rFGSM = MalwareDetectionModel().to(device)

# Load model parameters
model_AT_rFGSM.load_state_dict(torch.load('model_AT_rFGSM.pth', map_location=torch.device(device)))

<All keys matched successfully>

In [13]:
model_evaluation(model_AT_rFGSM,test_loader,device)

Test Accuracy: 0.9882
Test balanced Accuracy: 0.9619
Precision: 0.9713
Recall: 0.9274
F1-score: 0.9488
True Positives (TP): 1048
True Negatives (TN): 8401
False Positives (FP): 31
False Negatives (FN): 82
False Negative Rate (FNR): 7.2566
False Positive Rate (FPR): 0.3676


In [14]:
# rFGSM
attack_params = {"k":50, "epsilon":0.02, 'random':True, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, dfgsm_k, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, dfgsm_k, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 92.74% | Under attack: 92.04%.


In [32]:
def pgd(x, y, model, insertion_array, removal_array, k=25, step_length=0.02, norm='linf', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False):
    """
    Projected Gradient Descent (PGD) adversarial attack.
    :param y: Ground truth labels
    :param x: Feature vector
    :param model: Neural network model
    :param k: Number of steps
    :param step_length: Step size for each iteration
    :param norm: Norm used for perturbation ('linf' or 'l2')
    :param initial_rounding_threshold: Threshold parameter for rounding the initial x_next
    :param round_threshold: Threshold parameter for rounding
    :param random: Flag to generate random thresholds
    :param is_report_loss_diff: Flag to report loss difference
    :param is_sample: Flag to sample randomly from the feasible area
    :return: The adversarial version of x (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize starting point
    x_next = x.clone()
    x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

    # Multi-step PGD
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        gradients = grad_vars[0].data

        #pos_insertion = (x_var <= 0.5) * 1 * insertion_array
        pos_insertion = (x_var <= 0.5) * 1
        grad4insertion = (gradients >= 0) * pos_insertion * gradients

        pos_removal = (x_var > 0.5) * 1 * removal_array
        grad4removal = (gradients < 0) * pos_removal * gradients

        gradients = grad4removal + grad4insertion
        #gradients =  grad4insertion

        # Norm
        if norm == 'linf':
            perturbation = torch.sign(gradients).float()

        elif norm == 'l2':
            l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
            perturbation = (gradients / l2norm).float()
            #perturbation = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm).float()
            #perturbation = torch.maximum(torch.tensor(-1., dtype=x.dtype, device=x.device), perturbation).float()
            #perturbation[torch.isnan(perturbation)] = 0.
            #perturbation[torch.isinf(perturbation)] = 1.

        elif norm == 'l1':
            # consider just features of a sample which are not updated yet(because our update is 0to1 or 1to0 not stepwise)
            un_mod = torch.abs(x_next - x_var) <= 1e-6
            gradients = gradients * un_mod

            val, _ = torch.topk(torch.abs(gradients), 1)
            perturbation = (torch.abs(gradients) >= val.expand_as(gradients)).float() * torch.sign(gradients).float()
            # stop perturbing the examples that are successful to evade the victim
            outputs = model(x_next)
            _, predicted = torch.topk(outputs, k=1)
            done = (predicted != y).squeeze()
            if torch.all(done):
                break
            perturbation[done] = 0.

        else:
            raise ValueError("Expect 'l1' or 'l2' or 'linf' norm.")

        # Update x_next
        x_next = torch.clamp(x_next + perturbation * step_length, min=0., max=1.)

    # Rounding step
    if random:
       round_threshold = torch.rand(x_next.size())
    x_next = round_x(x_next, round_threshold=round_threshold)

    # Feasible projection
    #x_next = or_float_tensors(x_next, x)

    # Compute adversarial loss
    loss_adv = criterion(model(x_next), y.view(-1).long()).data

    if is_report_loss_diff:
        outputs = model(x_next)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"PGD {norm}: Attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    # Replace with natural if adversarial loss is higher
    replace_flag = (loss_adv < loss_natural).squeeze()
    x_next[replace_flag] = x[replace_flag]

    return x_next

In [33]:
# PGD-l2
attack_params = {'insertion_array':insertion_array, 'removal_array':removal_array,"k":50, "step_length":0.5, 'norm':'l2', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 92.74% | Under attack: 37.17%.


In [25]:
# PGD-linf
attack_params = {'insertion_array':insertion_array, 'removal_array':removal_array,"k":500, "step_length":0.002, 'norm':'linf', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 92.74% | Under attack: 42.04%.


In [27]:
# PGD-l1
attack_params = {'insertion_array':insertion_array, 'removal_array':removal_array,"k":500, "step_length":1., 'norm':'l1', 'random':False, 'is_report_loss_diff':False, 'is_sample':False}
adv_predict(test_loader, model_DNN, pgd, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, pgd, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 92.74% | Under attack: 36.55%.


In [ ]:
# Mimicry×10
attack_params = {"trials":10, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 25.31%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 94.16%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 95.04%.


In [ ]:
# Mimicry×30
attack_params = {"trials":30, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 25.31%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 93.63%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 94.34%.


In [ ]:
# Mimicry×100
attack_params = {"trials":100, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 24.6%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 92.39%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 92.3%.


In [ ]:
# Mimicry×1000
attack_params = {"trials":1000, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, mimicry, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, mimicry, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 2.478%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 92.04%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 87.26%.


In [ ]:
# PGD-max(MaxMA)
attack_params = {'steps_max':1,'is_sample':False, 'varepsilon':1e-20, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, PGD_Max2, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, PGD_Max2, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, PGD_Max2, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 89.29%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 80.62%.


In [ ]:
# PGD-max(iMaxMA)
attack_params = {'steps_max':5,'is_sample':False, 'varepsilon':1e-20, 'is_report_loss_diff':False}
adv_predict(test_loader, model_DNN, PGD_Max2, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM, PGD_Max2, device, **attack_params)
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, PGD_Max2, device, **attack_params)

Accuracy of just malwares (without attack): 96.19% | Under attack: 0.0%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 87.96%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 74.16%.


In [ ]:
attack_params = {'step_lengths':{"l1": 1.0, "l2": 0.5, "linf": 0.0008}, "steps":700}
adv_predict(test_loader, model_AT_rFGSM, StepwiseMax_onestep2, device, **attack_params)

step-wise max: attack effectiveness 7.826%.
step-wise max: attack effectiveness 8.547%.
step-wise max: attack effectiveness 15.200%.
step-wise max: attack effectiveness 6.796%.
step-wise max: attack effectiveness 11.504%.
step-wise max: attack effectiveness 16.216%.
step-wise max: attack effectiveness 9.630%.
step-wise max: attack effectiveness 9.489%.
step-wise max: attack effectiveness 7.200%.
step-wise max: attack effectiveness 12.245%.
Accuracy of just malwares (without attack): 94.69% | Under attack: 89.65%.


In [ ]:
attack_params = {'step_lengths':{"l1": 1.0, "l2": 0.2, "linf": 0.0005}, "steps":1000}
adv_predict(test_loader, model_AT_rFGSM_weightedLoss, StepwiseMax_onestep2, device, **attack_params)

step-wise max: attack effectiveness 9.565%.
step-wise max: attack effectiveness 7.692%.
step-wise max: attack effectiveness 16.800%.
step-wise max: attack effectiveness 10.680%.
step-wise max: attack effectiveness 12.389%.
step-wise max: attack effectiveness 20.721%.
step-wise max: attack effectiveness 11.852%.
step-wise max: attack effectiveness 10.949%.
step-wise max: attack effectiveness 8.800%.
step-wise max: attack effectiveness 12.245%.
Accuracy of just malwares (without attack): 96.64% | Under attack: 87.88%.


In [ ]:
# Define different attacks with their parameters
attacks = [
    (dfgsm_k, {"k":100, "epsilon":0.02, 'is_report_loss_diff' : False}),
    (bga_k, {"k":100, 'is_report_loss_diff' : False}),
    (bca_k, {"k":100, 'is_report_loss_diff' : False}),
    (grosse_k, {"k":100, 'is_report_loss_diff' : False}),

    (pgd, {'k': 100, 'step_length': 1., 'norm': 'l1', 'is_report_loss_diff' : False}),
    (pgd, {'k': 200, 'step_length': 0.05, 'norm': 'l2', 'is_report_loss_diff' : False}),
    (pgd, {'k': 100, 'step_length': 0.02, 'norm': 'linf', 'is_report_loss_diff' : False}),
    (StepwiseMax_onestep2, {'step_lengths':{"l1": 1.0, "l2": 0.05, "linf": 0.0008}, "steps":650}),

    # Add more attacks as needed
]

# Iterate over each attack and its parameters
for attack_func, attack_params in attacks:
    print(f"Running attack: {attack_func.__name__} with parameters: {attack_params}")
    adv_predict(test_loader, model_AT_rFGSM, attack_func, device, **attack_params)
    print()  # Print an empty line for separation


Running attack: dfgsm_k with parameters: {'k': 50, 'epsilon': 0.02, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 83.43%.

Running attack: bga_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 91.57%.

Running attack: bca_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.93%.

Running attack: grosse_k with parameters: {'k': 25, 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.93%.

Running attack: pgd with parameters: {'k': 100, 'step_length': 1.0, 'norm': 'l1', 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 87.24%.

Running attack: pgd with parameters: {'k': 200, 'step_length': 0.05, 'norm': 'l2', 'is_report_loss_diff': False}
Adversarial accuracy (without attack): 99.25% | Under attack: 91.67%.

Running att